In [1]:
# Imports
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import random
from datetime import datetime
import json
import training
import color
import vector as vec

In [2]:
# Constants

# Network Structure
CONTEXT_SIZE = 32       # How many other voxels are considered for a training example
EMBEDDING_SIZE = 520    # Dimensionality of the voxel embedding vector
STACKED_LAYERS = 1      # How many times the network structure repeats itself
ATTENTION_HEADS = 10    # Number of heads in each multi-headed attention mechanism
BATCH_SIZE = 128

# Training Hyperparameters
CHECK_RADIUS = 7        # How far away voxels can be to be part of a training example
CENTER_FOCUS = 0.3      # How much to focus on picking voxels close to the center of the cube. Must be between 0 and 1.
LEARNING_RATE = 1e-3
TRAINING_EXAMPLES = 1
AIR_WEIGHT = 9

In [3]:
# Load voxel palette
# The output is a 255-dimensional vector of probabilities for different colors
# Which 255 colors can be generated is decided by the palette file

# Index 0 is reserved as 'undecided' voxel
# Index 1 is reserved as 'air' voxel
# Index 2-255 are colors. So there are 254 possible colors.
with open('data/palette.json', 'r') as json_file:
    raw_palette = json.load(json_file)['colors']
    palette = color.expand_palette(raw_palette)
    palette_size = len(palette)

print(f"Palette has {palette_size} colors")

Palette has 256 colors


In [4]:
# Set up loss and optimizer
def weighted_categorical_crossentropy():
    weights = np.ones(palette_size-1).astype(np.float64) 
    weights[0] = AIR_WEIGHT
    weights = tf.keras.backend.variable(weights, dtype=tf.float64)
    weights_tiled = np.tile(weights, (BATCH_SIZE, CONTEXT_SIZE, 1))

    loss_object = tf.losses.CategoricalCrossentropy()

    # def loss(y_true, y_pred):
    #     # Convert tensors to float64 for consistent data type operations
    #     y_true = tf.cast(y_true, tf.float64)
    #     y_pred = tf.cast(y_pred, tf.float64)
        
    #     # scale predictions so that the class probas of each sample sum to 1
    #     y_pred /= tf.keras.backend.sum(y_pred, axis=-1, keepdims=True)
    #     # clip to prevent NaN's and Inf's
    #     y_pred = tf.keras.backend.clip(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
    #     # calculate loss and weight it
    #     loss = y_true * tf.keras.backend.log(y_pred) * weights
    #     loss = -tf.keras.backend.sum(loss, -1)
    #     return loss

    #     # return tf.keras.losses.categorical_crossentropy(y_true, y_pred) * (y_true * weights)

    def loss(y_true, y_pred):
        # loss = loss_object(y_true=y_true, y_pred=y_pred, sample_weight=weights_tiled)
        loss = loss_object(y_true=y_true, y_pred=y_pred)
        return loss

    return loss

# Use custom loss with the class weights
loss_function = weighted_categorical_crossentropy()
# loss_function = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)

In [5]:
# Create model
# def main_model():
#     input = keras.Input(shape=(CONTEXT_SIZE, EMBEDDING_SIZE,), name='input')
#     input_next_pos = keras.Input(shape=(CONTEXT_SIZE, EMBEDDING_SIZE,), name='input_next_pos')

#     # Normalization
#     x = keras.layers.LayerNormalization(name=f'normalization_start_a')(input)

#     # Stacked layers
#     for i in range(STACKED_LAYERS):
#         # Multi-headed attention
#         fx = keras.layers.MultiHeadAttention(
#             num_heads=ATTENTION_HEADS,
#             key_dim=EMBEDDING_SIZE,
#             name=f'multi_head_attention_{i}',
#         )(x, x, use_causal_mask=True)

#         # Residual connection
#         x = keras.layers.Add(name=f'residual_connection_{i}a')([x,fx])

#         # Normalization
#         x = keras.layers.LayerNormalization(name=f'normalization_{i}a')(x)

#         # Feedforward
#         fx = keras.layers.Dense(EMBEDDING_SIZE, name=f'feedforward_{i}')(x)
#         fx = keras.layers.LeakyReLU(name=f'relu_{i}')(fx)

#         # Residual connection
#         x = keras.layers.Add(name=f'residual_connection_{i}b')([x,fx])

#         # Normalization
#         x = keras.layers.LayerNormalization(name=f'normalization_{i}b')(x)
    
#     # Concatenate with next_pos input
#     x = keras.layers.Concatenate(axis=2, name='concatenate_next_pos')([x,input_next_pos])

#     # Final feedforward layer
#     # Output size should be palette_size-1, since we don't want it to be able to choose "undecided"
#     x = keras.layers.Dense(palette_size-1, name='feedforward_final')(x)

#     # Softmax
#     x = keras.layers.Softmax(name='softmax')(x)
    
#     # Build and return model
#     return keras.Model(inputs=[input, input_next_pos], outputs=x)

def main_model():
    input = keras.Input(shape=(CONTEXT_SIZE, EMBEDDING_SIZE,), name='input')
    input_next_pos = keras.Input(shape=(CONTEXT_SIZE, EMBEDDING_SIZE,), name='input_next_pos')

    # Normalization
    x = keras.layers.LayerNormalization(name=f'normalization_start_a')(input)
    
    # Concatenate with next_pos input
    x = keras.layers.Concatenate(axis=2, name='concatenate_next_pos')([x,input_next_pos])

    # Final feedforward layer
    # Output size should be palette_size-1, since we don't want it to be able to choose "undecided"
    x = keras.layers.Dense(palette_size-1, name='feedforward_final')(x)

    # Softmax
    x = keras.layers.Softmax(name='softmax')(x)
    
    # Build and return model
    return keras.Model(inputs=[input, input_next_pos], outputs=x)

model = main_model()
model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 32, 520)]    0           []                               
                                                                                                  
 normalization_start_a (LayerNo  (None, 32, 520)     1040        ['input[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 input_next_pos (InputLayer)    [(None, 32, 520)]    0           []                               
                                                                                                  
 concatenate_next_pos (Concaten  (None, 32, 1040)    0           ['normalization_start_a[0][0]

In [6]:
# Design training examples
training_examples = training.generate_training_examples(TRAINING_EXAMPLES, CONTEXT_SIZE)

print(f"{len(training_examples)} training examples created.")

print(random.choice(training_examples))

1 training examples created.
[((0, 1, 0), 246), ((1, 1, 0), 231), ((2, 1, 0), 246), ((3, 1, 0), 246), ((4, 1, 0), 231), ((0, 2, 0), 246), ((1, 2, 0), 246), ((2, 2, 0), 231), ((3, 2, 0), 246), ((4, 2, 0), 246), ((0, 3, 0), 231), ((1, 3, 0), 246), ((2, 3, 0), 246), ((3, 3, 0), 231), ((4, 3, 0), 246), ((0, 4, 0), 246), ((1, 4, 0), 231), ((2, 4, 0), 246), ((3, 4, 0), 246), ((4, 4, 0), 231), ((0, 0, 1), 246), ((1, 0, 1), 246), ((2, 0, 1), 252), ((3, 0, 1), 1), ((4, 0, 1), 1), ((0, 1, 1), 1), ((1, 1, 1), 1), ((2, 1, 1), 1), ((3, 1, 1), 1), ((4, 1, 1), 1), ((0, 2, 1), 1), ((1, 2, 1), 1), ((2, 2, 1), 1)]


In [7]:
def encode_training_input(example):
    inputEntry = []

    # Encode context vector
    for index, voxel in enumerate(example):
        if len(inputEntry) < CONTEXT_SIZE+1:
            inputEntry.append(training.embed(index, voxel[0], voxel[1], palette, EMBEDDING_SIZE))

    # Pad remainder of context with zeros
    if len(inputEntry) < CONTEXT_SIZE+1:
        zero_elem = [0,] * EMBEDDING_SIZE
        inputEntry += [zero_elem,] * ((CONTEXT_SIZE+1) - len(inputEntry))

    return inputEntry

def encode_training_input_next_pos(example, last_pos):
    inputEntry = []

    # Encode context vector
    for index in range(len(example)):
        pos = example[index][0]
        if index < len(example)-1:
            next_pos = example[index+1][0]
        else:
            next_pos = last_pos
        if len(inputEntry) < CONTEXT_SIZE+1:
            # inputEntry.append(training.embed_nptest(index, vec.subtract(next_pos, pos), -1, palette, EMBEDDING_SIZE))
            inputEntry.append(training.embed_nptest(index, next_pos, -1, palette, EMBEDDING_SIZE))

    # Pad remainder of context with zeros
    if len(inputEntry) < CONTEXT_SIZE+1:
        zero_elem = [0,] * EMBEDDING_SIZE
        inputEntry += [zero_elem,] * ((CONTEXT_SIZE+1) - len(inputEntry))
    
    return inputEntry

def encode_training_output(example):
    outputEntry = []
    for voxel in example:
        outputEntry.append(training.encode_one_hot(voxel[1], palette_size))
    return outputEntry

# Reformat training examples into tensor format
def encode_training_examples(training_examples):
    training_inputs = []
    training_inputs_next_pos = []
    training_outputs = []
    for example in training_examples:
        input_entry = encode_training_input(example)
        input_entry_next_pos = encode_training_input_next_pos(example, (0, 0, 0))
        output_entry = encode_training_output(example)
        
        # Shift the output
        input_entry.pop(-1)
        input_entry_next_pos.pop(-1)
        output_entry.pop(0)

        # Push to training example list
        training_inputs.append(input_entry)
        training_inputs_next_pos.append(input_entry_next_pos)
        training_outputs.append(output_entry)

    training_input_tensor = tf.Variable(training_inputs, tf.float64)
    training_input_next_pos_tensor = tf.Variable(training_inputs_next_pos, tf.float64)
    training_output_tensor = tf.Variable(training_outputs, tf.float64)

    return training_input_tensor, training_input_next_pos_tensor, training_output_tensor

training_input_tensor, training_input_next_pos_tensor, training_output_tensor = encode_training_examples(training_examples)

In [8]:
# print(training_input_next_pos_tensor[12][0])

# print(len(training_input_tensor[0][0]))
# print(len(training_input_next_pos_tensor[0][0]))
# print(len(training_output_tensor[0][0]))


# i = 17
# position = training_examples[0][i][0]
# print(position[0] + (position[1] * 5) + (position[2] * 5 * 5))
# print(training_examples[0][i+1])
# print(training_input_next_pos_tensor[0][i])
# for j in range(EMBEDDING_SIZE):
#     if int(training_input_next_pos_tensor[0][i][j]) > 0:
#         print(j)
#         break

# for j in range(EMBEDDING_SIZE):
#     if int(training_output_tensor[0][21][j]) > 0:
#         print(j+1)
#         break


In [9]:
# Batch training data
dataset_input = tf.data.Dataset.from_tensor_slices(training_input_tensor)
dataset_input_next_pos = tf.data.Dataset.from_tensor_slices(training_input_next_pos_tensor)
dataset_output = tf.data.Dataset.from_tensor_slices(training_output_tensor)

training_input_batched = dataset_input.batch(BATCH_SIZE)
training_input_next_pos_batched = dataset_input_next_pos.batch(BATCH_SIZE)
training_output_batched = dataset_output.batch(BATCH_SIZE)

In [10]:
def pick_weighted_random(probabilities):
    choice = random.random()
    for i in range(len(probabilities)):
        choice -= probabilities[i]
        if choice < 0:
            return i + 1
    return 1

def pick_highest(probabilities):
    chosen_voxel = 1
    best = 0
    for i in range(len(probabilities)):
        if probabilities[i] > best:
            best = probabilities[i]
            chosen_voxel = i+1
    
    return chosen_voxel

def pick_voxel_from_probabilities(probabilities):
    return pick_weighted_random(probabilities)

# Construct an example sculpture using the model's current progress
# TODO: Move this to a separate file so we can do multithreading and other improvements
def build_sculpture(count, base=None, temperature=1.0, debug=False):
    # If a base wasn't specified, create one
    color = int(random.random() * 254) + 1
    voxels = {}
    start_pos = (0, 0, 0)
    # start_pos = (int(random.random() * training.SIZE[0]), int(random.random() * training.SIZE[1]), int(random.random() * training.SIZE[2]))
    voxels[vec.ttos(start_pos)] = color
    
    # Build context vector for the sculpture
    context = [(start_pos, color)]

    for i in range(count-1):
        # Determine where the next voxel will go
        # TODO: Encode this data into the model somehow
        next_pos = training.pick_next_voxel(voxels, context)

        # Get the output from the model
        input_data = encode_training_input(context)
        input_data.pop(-1)
        input_tensor = tf.Variable(input_data, tf.float64)
        input_tensor = tf.reshape(input_tensor, [1, -1, EMBEDDING_SIZE])

        input_next_pos_data = encode_training_input_next_pos(context, next_pos)

        if debug:
            print("Next Pos indices: ", end=" ")
            for i in range(len(input_next_pos_data)):
                for j in range(EMBEDDING_SIZE):
                    if int(input_next_pos_data[i][j]) > 0:
                        print(j, end=" ")
                        break
                    if j > 255:
                        print("No", end=" ")
                        break
            print()

        input_next_pos_data.pop(-1)
        input_next_pos_tensor = tf.Variable(input_next_pos_data, tf.float64)
        input_next_pos_tensor = tf.reshape(input_tensor, [1, -1, EMBEDDING_SIZE])

        output = model([input_tensor, input_next_pos_tensor], training=False)
        if debug:
            print(f"Context index: {len(context)-1}")
            print(f"Context at index: {context[len(context)-1]}")
        output_probabilities = output[0][len(context)-1]

        # Pick which voxel to generate based on output probabilities
        # TODO: Implement temperature
        chosen_voxel = pick_voxel_from_probabilities(output_probabilities)

        if debug:
            print(f"Chosen voxel: {float(chosen_voxel)}")
        
        # Build the voxel
        voxels[vec.ttos(next_pos)] = chosen_voxel
        context.append((next_pos, chosen_voxel))
        if len(context) > CONTEXT_SIZE:
            context = training.remove_farthest(context, next_pos)
        
        if debug:
            print()
        
    return voxels

def build_sculpture_test(count, base=None, temperature=1.0, debug=True):
    te = training_examples[0]

    # If a base wasn't specified, create one
    color = te[0][1]
    voxels = {}
    start_pos = te[0][0]
    # start_pos = (int(random.random() * training.SIZE[0]), int(random.random() * training.SIZE[1]), int(random.random() * training.SIZE[2]))
    voxels[vec.ttos(start_pos)] = color
    
    # Build context vector for the sculpture
    context = [(start_pos, color)]

    for i in range(min(count-1, CONTEXT_SIZE)):
        # Determine where the next voxel will go
        # TODO: Encode this data into the model somehow
        next_pos = te[i+1][0]

        # Get the output from the model
        input_data = encode_training_input(context)
        input_data.pop(-1)
        input_tensor = tf.Variable(input_data, tf.float64)
        input_tensor = tf.reshape(input_tensor, [1, -1, EMBEDDING_SIZE])

        input_next_pos_data = encode_training_input_next_pos(context, next_pos)

        input_next_pos_data.pop(-1)
        input_next_pos_tensor = tf.Variable(input_next_pos_data, tf.float64)
        input_next_pos_tensor = tf.reshape(input_tensor, [1, -1, EMBEDDING_SIZE])

        output = model([input_tensor, input_next_pos_tensor], training=False)
        output_probabilities = output[0][len(context)-1]

        # Pick which voxel to generate based on output probabilities
        # TODO: Implement temperature
        chosen_voxel = pick_voxel_from_probabilities(output_probabilities)
        correct_voxel = te[i+1][1]

        if chosen_voxel == correct_voxel:
            print(f"Correct Inference: {chosen_voxel}")
        else:
            print(f"Incorrect: {chosen_voxel} should be {correct_voxel}")
        
        # Build the voxel
        voxels[vec.ttos(next_pos)] = correct_voxel
        context = te[:i+1]
        
    return voxels

build_sculpture(15, debug=True)

Next Pos indices:  1 No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No 
Context index: 0
Context at index: ((0, 0, 0), 229)
Chosen voxel: 179.0

Next Pos indices:  1 2 No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No 
Context index: 1
Context at index: ((1, 0, 0), 179)
Chosen voxel: 134.0

Next Pos indices:  1 2 3 No No No No No No No No No No No No No No No No No No No No No No No No No No No No No No 
Context index: 2
Context at index: ((2, 0, 0), 134)
Chosen voxel: 202.0

Next Pos indices:  1 2 3 4 No No No No No No No No No No No No No No No No No No No No No No No No No No No No No 
Context index: 3
Context at index: ((3, 0, 0), 202)
Chosen voxel: 196.0

Next Pos indices:  1 2 3 4 5 No No No No No No No No No No No No No No No No No No No No No No No No No No No No 
Context index: 4
Context at index: ((4, 0, 0), 196)
Chosen voxel: 19.0

Next Pos indices:  1 2 3 4 5 6 No No No No No No No No N

{'0,0,0': 229,
 '1,0,0': 179,
 '2,0,0': 134,
 '3,0,0': 202,
 '4,0,0': 196,
 '0,1,0': 19,
 '1,1,0': 248,
 '2,1,0': 17,
 '3,1,0': 53,
 '4,1,0': 129,
 '0,2,0': 98,
 '1,2,0': 222,
 '2,2,0': 107,
 '3,2,0': 222,
 '4,2,0': 222}

In [11]:
# Function for saving sculptures to json
def save_sculpture(sculpture, filename):
    json_data = {
        "size": {
            "x": training.SIZE[0],
            "y": training.SIZE[1],
            "z": training.SIZE[2],
        },
        "voxels": sculpture,
    }
    with open(filename, 'w') as output_file:
        json.dump(json_data, output_file, indent=2)

In [12]:
# Training step
@tf.function
def train_step(input_data, input_next_pos_data, output_data):
    # Set up tape
    with tf.GradientTape() as tape:
      output = model([input_data, input_next_pos_data], training=True)

      loss = loss_function(output_data, output)

      gradients = tape.gradient(loss, model.trainable_variables)

      optimizer.apply_gradients(zip(gradients, model.trainable_variables))

      return loss

# Training loop
def train(epochs):
  time_started = datetime.now()
  example_time = 3
  sculptures_made = 0

  # Epochs
  for epoch in range(epochs):
    # Minibatches
    for (batch_input, batch_input_next_pos, batch_output) in zip(training_input_batched, training_input_next_pos_batched, training_output_batched):
      train_step(batch_input, batch_input_next_pos, batch_output)
      # print(float(train_step(batch_input, batch_input_next_pos, batch_output)))

    # Print status
    # print(f"Completed Epoch {epoch}")

    # Check if we should output an example sculpture
    if (datetime.now()-time_started).total_seconds() >= example_time:
      print(f"Epoch {epoch}")
      print("Building example sculpture...")
      sculpture = build_sculpture(training.SIZE[0] * training.SIZE[1] * training.SIZE[2])
      # sculpture = build_sculpture(300)
      sculptures_made += 1
      sculpture_filename = f"examples/json/example_{(time_started-datetime.utcfromtimestamp(0)).total_seconds()}_{sculptures_made}.json"
      save_sculpture(sculpture, sculpture_filename)
      print("Done")
      example_time = (datetime.now()-time_started).total_seconds() * 1.3

train(10000000)

Epoch 525
Building example sculpture...
Done
Epoch 973
Building example sculpture...
Done
Epoch 1853
Building example sculpture...
Done
Epoch 3277
Building example sculpture...
Done
Epoch 5398
Building example sculpture...
Done
Epoch 8448
Building example sculpture...
Done
Epoch 12714
Building example sculpture...
Done
Epoch 18308
Building example sculpture...
Done
Epoch 26120
Building example sculpture...
Done
Epoch 36525
Building example sculpture...
Done


KeyboardInterrupt: 

In [ ]:
model.save('nextpostest.h5')